<a href="https://colab.research.google.com/github/cassbech/NLP_sentiment_analysis/blob/master/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJET 10H11 - Analyse de Sentiment

Voici un template pour l'import de fichier excel, la transformation en matrice de termes et l'application de différents modèle de machine learning.

Pour le bon fonctionnement du template vérifier que tous la base de donnée excel et la bibliotèque soient dans le même dossier que ce template.

# Installation des modules indispensables

In [0]:
# Enlever les # des lignes en dessous si ces modules ne sont pas encore installés sur votre ordinateur
# ! pip install unidecode
# ! pip install unicode

# Importer notre librairie

In [0]:
from SentimentAnalysisBibliotheque import *

[nltk_data] Downloading package punkt to /Users/mac.admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mac.admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importer notre dataframe et la transformer en matrice de termes

### Les dataframes générés avec le scraping contiennent au moins une colonne tweets

In [0]:
def main():
    # data_tweet_sncf.xlsx
    fichier = input("Quel est le nom complet de votre fichier au format excel : Attention à ne pas mettre de guillemets, exemple : data_tweet_sncf.xlsx . Votre réponse : " )
    nombre = input('Si vous disposez d\'une colonne \"users\" en plus de vos colonnes \"tweets\" et \"tonalité\" tapez 3 sinon tapez 2 : ')
    pub = input('Si vous disposez d\'une colonne \"pub/presse\" en plus de vos colonnes \"tweets\" et \"tonalité\" tapez oui sinon tapez non  : ')
    
    if (nombre == "3"):
        if (pub=="oui"):
            data_frame = pd.read_excel(fichier,usecols=["users","tweets",'tonalité',"pub/presse"])
            data_frame["All_tweets"] = data_frame["tweets"].copy()
            data_frame.rename(index=str, columns={"users": "Users", "tweets": "Tweets traités","tonalité":"Tonalité","pub/presse":"Pub"},inplace=True)
            users = data_frame["Users"].tolist()
        else:
            data_frame = pd.read_excel(fichier,usecols=["users","tweets",'tonalité'])
            data_frame["All_tweets"] = data_frame["tweets"].copy()
            data_frame.rename(index=str, columns={"users": "Users", "tweets": "Tweets traités","tonalité":"Tonalité"},inplace=True)
            users = data_frame["Users"].tolist() 
    else:
        if (pub=="oui"):
            data_frame = pd.read_excel(fichier,usecols=["tweets","tonalité","pub/presse"])
            data_frame["All_tweets"] = data_frame["tweets"].tolist()
            data_frame.rename(index=str, columns={"tweets": "Tweets traités","tonalité":"Tonalité","pub/presse":"Pub"},inplace=True)
        else:
            data_frame = pd.read_excel(fichier,usecols=["tweets","tonalité"])
            data_frame["All_tweets"] = data_frame["tweets"].tolist()
            data_frame.rename(index=str, columns={"tweets": "Tweets traités","tonalité":"Tonalité"},inplace=True)

    data_frame["Tweets traités"] = data_frame["Tweets traités"].str.lower()
    data_frame["Tweets traités"] = data_frame["Tweets traités"].astype(str)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(remove_url)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(filter_tab)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(remove_at)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(remove_ponctuation)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(remove_stop_words)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(remplace_accents)
    data_frame["Tweets traités"] = data_frame["Tweets traités"].apply(remove_ponctuation)
    
    list_of_words = word_occurency(data_frame)
    result = matrix_of_termes(data_frame,list_of_words)
    
    if (nombre == "3"):
        result["Users"] = users
        
    volontee = input("Voulez-vous télécharger le résultat au format csv ? : y / n : ")
    if (volontee == "y"):
        create_data_frame(result)
    
    if(pub=="non"):
        result["Pub"]=0
        result["Pub"].loc[result[result["Tonalité"].isnull()].index.tolist()]=1
        result["Tonalité"]=result["Tonalité"].fillna("2")
        result["Tonalité"].replace(" ","2")
        cols = result.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        result = result[cols]
        
    if (pub =="oui"):
        result["Pub"] = result["Pub"].fillna("0")
    
    result["Tonalité"] = result["Tonalité"].astype(int)
    result["Pub"] = result["Pub"].astype(int)
    
    return(result)

In [0]:
if __name__ == '__main__':
    resultat = main()

Quel est le nom complet de votre fichier au format excel : Attention à ne pas mettre de guillemets, exemple : data_tweet_sncf.xlsx . Votre réponse : RATP_tweets.xlsx
Si vous disposez d'une colonne "users" en plus de vos colonnes "tweets" et "tonalité" tapez 3 sinon tapez 2 : 3
Voulez-vous télécharger le résultat au format csv ? : y / n : n


**Aperçu de la matrice de termes**

In [0]:
resultat

,Tweets traités,Tonalité,All_tweets,de,la,ratp,a,le,est,et,...,froc,soufflet,lecteur,verbalisable,ledru,bloc,defenfra,fanatises,useless,aaah
0,effectivement ca s est passe entre trocadero e...,2,@ClientsRATP @antoine_HD_ @eN_Drop @Ligne6_RAT...,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,merci mais les pages du site ratp ne sont pas...,2,@ClientsRATP Merci. Mais les pages du site RAT...,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,au tour de haylen au top,2,"Au tour de Haylen, au top 🥁 https://t.co/Krtxt...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,salut il y a un groupe qui fait du proselytism...,2,Salut @Ligne2_RATP il y a un groupe qui fait d...,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,vous connaissez hugo barriol on adore,2,Vous connaissez Hugo Barriol ? On adore ! 😍 ht...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,et pas des files partagees avec des poi...,2,@ParisEnVoiture et 👏 pas 👏 des 👏 files 👏 parta...,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,c est parti pour shoefiti,2,C'est parti pour Shoefiti ! 🎧 https://t.co/vHB...,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,si vous etes bien a la ratp mais nous gerons ...,2,@PYEtienney Si vous êtes bien à la RATP. Mais ...,1,2,1,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,les derives islamistes ne sont plus tolerables...,1,Les dérives islamistes ne sont plus tolérables...,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,elles ne pourraient pas monter dans un bus de ...,2,@29_suzane @gelleejeanluc Elles ne pourraient ...,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Répartition des tonalités**

In [0]:
tona = resultat.groupby(["Tonalité"]).count()
tona["Tweets traités"]
# 1: Négatif
# 2: Neutre
# 3: Positif

Tonalité
1    1489
2    3939
3     238
Name: Tweets traités, dtype: int64

# Importation des modèles 

In [0]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.svm import LinearSVC
from sklearn import naive_bayes

# Division du Dataset

Ici on drop la colonne **'Users'** car cette colonne est présente dans le fichier excel de base. Ceci n'est pas toujours le cas! Si cette colonne est absente, la supprimer de la ligne de code resultat.drop()

In [0]:
# En X nous prenons la matrice de termes uniquement.
X = resultat.drop(["Users", "Tweets traités", "Tonalité", "All_tweets"], axis = 1)
# En y nous prenons la Tonalité des tweets.
y = resultat["Tonalité"]

In [0]:
# Training set: 80%  Testing set: 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Application des modèles 

**Decision Tree**

In [0]:
# DECISION TREE
dectree = DecisionTreeClassifier()
dectree.fit(X_train, y_train)
y_pred = dectree.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))

# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Le score de précision est de 67.55 %
260 tweets négatifs.
836 tweets neutres.
38 tweets positifs.


**Random Forest**

In [0]:
# RANDOM FOREST
print("Random Forest")
forest = RandomForestClassifier(n_estimators=100, max_features="auto", random_state=0)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))


# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Le score de précision est de 73.19 %
57 tweets négatifs.
1060 tweets neutres.
17 tweets positifs.


**Bagging Classifier**

In [0]:
# BAGGING CLASSIFIER
print("Bagging Classifier")
bag = BaggingClassifier(n_estimators=100)
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))

# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Le score de précision est de 74.07 %
149 tweets négatifs.
963 tweets neutres.
22 tweets positifs.


**Adaptive Boosting**

In [0]:
# ADABOOST
print("Adaptive Boosting aka Adaboost")
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))

# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Adaptive Boosting aka Adaboost
Le score de précision est de 71.34 %
164 tweets négatifs.
942 tweets neutres.
28 tweets positifs.


**Gradient Boosting** 

In [0]:
# GRADIENT BOOSTING 
print("Gradient Boosting")
gbo = GradientBoostingClassifier(n_estimators=100)
gbo.fit(X_train, y_train)
y_pred = gbo.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))

# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Gradient Boosting
Le score de précision est de 73.19 %
116 tweets négatifs.
1002 tweets neutres.
16 tweets positifs.


**Support Vector Machine**

In [0]:
# SUPPORT VECTOR MACHINE
print("Support Vector Machine")

svm=LinearSVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))

# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Support Vector Machine
Le score de précision est de 76.28 %
260 tweets négatifs.
839 tweets neutres.
35 tweets positifs.


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


**Naive Bayes**

In [0]:
# NAIVE BAYES
print("Naive Bayes")
naive = naive_bayes.MultinomialNB()
naive.fit(X_train, y_train)
y_pred = naive.predict(X_test)

# Impression du score de précision
print("Le score de précision est de {} %".format(round((accuracy_score(y_test, y_pred)*100), 2)))

# Répartition des tonalités
print("{} tweets négatifs.".format(np.count_nonzero(y_pred == 1)))
print("{} tweets neutres.".format(np.count_nonzero(y_pred == 2))) 
print("{} tweets positifs.".format(np.count_nonzero(y_pred == 3)))

Naive Bayes
Le score de précision est de 73.99 %
407 tweets négatifs.
724 tweets neutres.
3 tweets positifs.
